## Evaluation

Batch prediction and evaluation are very similar. They are based on DataFlow pipeline and CloudML provides Evaluate and Prediction DataFlow transform. Datalab can generate DataFlow pipeline code template for you, just like Preprocessing.

Run "%ml evaluate" to generate input cell.

In [ ]:
%ml evaluate

After fill in the required fields, run it to generate code.

In [ ]:
%%ml evaluate
preprocessed_eval_data_path: /content/datalab/tmp/ml/iris/preprocessed/features_eval
metadata_path: /content/datalab/tmp/ml/iris/preprocessed/metadata.yaml
model_dir: /content/datalab/tmp/ml/iris/model/model
output_dir: /content/datalab/tmp/ml/iris/evaluate
output_prediction_name: predictions


Run the generated code. Optionally uncomment the code for creating confusion matrix plot. Note that the confusionmatrix code is only generated in classification case. 

In [3]:

# header
"""
Following code is generated from command line:
%%ml evaluate
preprocessed_eval_data_path: /content/datalab/tmp/ml/iris/preprocessed/features_eval
metadata_path: /content/datalab/tmp/ml/iris/preprocessed/metadata.yaml
model_dir: /content/datalab/tmp/ml/iris/model/model
output_dir: /content/datalab/tmp/ml/iris/evaluate
output_prediction_name: predictions

Please modify as appropriate!!!
"""

# imports
import apache_beam as beam
from apache_beam.io import fileio
import google.cloud.ml as ml
import google.cloud.ml.analysis as analysis
import google.cloud.ml.dataflow.io.tfrecordio as tfrecordio
import google.cloud.ml.io as io
import json
import os

# defines
def extract_values((example, prediction)):
  import tensorflow as tf
  tf_example = tf.train.Example()
  tf_example.ParseFromString(example.values()[0])
  feature_map = tf_example.features.feature
  values = {'target': feature_map['species'].int64_list.value[0]}
  values.update(prediction)
  return values

OUTPUT_DIR = '/content/datalab/tmp/ml/iris/evaluate'
pipeline = beam.Pipeline('DirectPipelineRunner')


# evaluation
eval_data_source = tfrecordio.TFRecordSource('/content/datalab/tmp/ml/iris/preprocessed/features_eval', compression_type=fileio.CompressionTypes.ZLIB)
eval_features = pipeline | eval_data_source
trained_model = pipeline | io.LoadModel('LoadModel', '/content/datalab/tmp/ml/iris/model/model')
evaluations = (eval_features | ml.Evaluate(trained_model, label='Evaluate')
    | beam.Map('ExtractEvaluationResults', extract_values))
eval_data_sink = beam.io.TextFileSink(os.path.join(OUTPUT_DIR, 'eval'))
evaluations | beam.Write('WriteEval', eval_data_sink)

# analysis
def make_data_for_analysis(values):
  return {
      'target': values['target'],
      'predicted': values['predictions'],
      'score': 0.0,
  }

metadata = pipeline | io.LoadMetadata('/content/datalab/tmp/ml/iris/preprocessed/metadata.yaml')
analysis_source = evaluations | beam.Map('CreateAnalysisSource', make_data_for_analysis)
confusion_matrix, precision_recall, logloss = (analysis_source |
    analysis.AnalyzeModel('Analyze Model', metadata))
confusion_matrix_file = os.path.join(OUTPUT_DIR, 'analyze_cm.json')
confusion_matrix_sink = beam.io.TextFileSink(confusion_matrix_file, shard_name_template='')
confusion_matrix | beam.io.Write('WriteConfusionMatrix', confusion_matrix_sink)

# run pipeline
pipeline.run()

# View Confusion Matrix with the following code:
#
import datalab.ml
import yaml
with ml.util._file.open_local_or_gcs(confusion_matrix_file, 'r') as f:
  data = [yaml.load(line) for line in f.read().rstrip().split('\n')]
datalab.ml.ConfusionMatrix([d['predicted'] for d in data],
                           [d['target'] for d in data],
                           [d['count'] for d in data]).plot()


To generate a pipeline that runs in cloud, simply run "%ml evaluate --cloud". Also all paths need to be GCS paths.

In [10]:
import os

bucket = 'gs://' + datalab_project_id() + '-sampledata'
eval_data_path = os.path.join(bucket, 'iris', 'preprocessed', 'features_eval')
metadata_path = os.path.join(bucket, 'iris', 'preprocessed', 'metadata.yaml')
model_path = os.path.join(bucket, 'iris', 'trained', 'model')
output_dir = os.path.join(bucket, 'iris', 'evaluate')

In [ ]:
%%ml evaluate --cloud
preprocessed_eval_data_path: $eval_data_path
metadata_path: $metadata_path
model_dir: $model_path
output_dir: $output_dir

In [13]:

# header
"""
Following code is generated from command line:
%%ml evaluate --cloud
preprocessed_eval_data_path: $eval_data_path
metadata_path: $metadata_path
model_dir: $model_path
output_dir: $output_dir

Please modify as appropriate!!!
"""

# imports
import apache_beam as beam
from apache_beam.io import fileio
import google.cloud.ml as ml
import google.cloud.ml.analysis as analysis
import google.cloud.ml.dataflow.io.tfrecordio as tfrecordio
import google.cloud.ml.io as io
import json
import os

# defines
def extract_values((example, prediction)):
  import tensorflow as tf
  tf_example = tf.train.Example()
  tf_example.ParseFromString(example.values()[0])
  feature_map = tf_example.features.feature
  values = {'target': feature_map['species'].int64_list.value[0]}
  values.update(prediction)
  return values

OUTPUT_DIR = 'gs://cloud-ml-test-automated-sampledata/iris/evaluate'
import datetime
options = {
    'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
    'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
    'job_name': 'evaluate' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S'),
    'project': 'cloud-ml-test-automated',
    'extra_packages': ['gs://cloud-ml/sdk/cloudml-0.1.2.latest.tar.gz'],
    'teardown_policy': 'TEARDOWN_ALWAYS',
    'no_save_main_session': True
}
opts = beam.pipeline.PipelineOptions(flags=[], **options)
pipeline = beam.Pipeline('DataflowPipelineRunner', options=opts)


# evaluation
eval_data_source = tfrecordio.TFRecordSource('gs://cloud-ml-test-automated-sampledata/iris/preprocessed/features_eval', compression_type=fileio.CompressionTypes.ZLIB)
eval_features = pipeline | eval_data_source
trained_model = pipeline | io.LoadModel('LoadModel', 'gs://cloud-ml-test-automated-sampledata/iris/trained/model')
evaluations = (eval_features | ml.Evaluate(trained_model, label='Evaluate')
    | beam.Map('ExtractEvaluationResults', extract_values))
eval_data_sink = beam.io.TextFileSink(os.path.join(OUTPUT_DIR, 'eval'))
evaluations | beam.Write('WriteEval', eval_data_sink)

# analysis

# run pipeline
pipeline.run()


<DataflowPipelineResult <Job
 id: u'2016-08-31_22_41_30-12572926462340872569'
 projectId: u'cloud-ml-test-automated'
 steps: []
 tempFiles: []
 type: TypeValueValuesEnum(JOB_TYPE_BATCH, 1)> at 0x7f5b606f0ed0>